## Define functions

In [5]:
# Import libraries

import pandas as pd
import numpy as np
import os
path = r'C:\Users\User\Instacart Basket Analysis'
df_orders_products_merge=pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

In [6]:
# Subsampling
df = df_orders_products_merge[:1000000]

In [7]:
# Define 'price_label' function
def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return np.nan

In [8]:
# Use this function 
df['price_range'] = df.apply(price_label, axis = 1)

C:\Users\User\AppData\Local\Temp\ipykernel_13188\2890558116.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df.apply(price_label, axis = 1)


In [9]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    652638
Low-range product    338018
High range             9344
Name: count, dtype: int64

In [10]:
df['prices'].max()

24.5

In [11]:
# Same using loc 
df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'

C:\Users\User\AppData\Local\Temp\ipykernel_13188\1399958827.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [12]:
df.loc[(df['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [13]:
df.loc[df['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [14]:
df['price_range'].value_counts(dropna = False)

price_range
Mid-range product    652638
Low-range product    338018
High range             9344
Name: count, dtype: int64

### Setting flags for a full dataframe

In [15]:
df_orders_products_merge.loc[df_orders_products_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [16]:
df_orders_products_merge.loc[(df_orders_products_merge['prices'] <= 15) & (df_orders_products_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [17]:
df_orders_products_merge.loc[df_orders_products_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [18]:
df_orders_products_merge['price_range_loc'].value_counts(dropna = False)

price_range_loc
Mid-range product     21860860
Low-range product     10126321
High-range product      417678
Name: count, dtype: int64

In [19]:
df_orders_products_merge.rename(columns = {'order_dow' : 'orders_day_of_the_week'}, inplace = True) 

In [20]:
df_orders_products_merge.columns

Index(['Unnamed: 0_x', 'product_id', 'product_name', 'aisle_id',
       'department_id', 'prices', 'Unnamed: 0_y', 'order_id', 'user_id',
       'order_number', 'orders_day_of_the_week', 'order_hour_of_day',
       'days_since_prior_order', 'days_since_prior_order_Flag',
       'add_to_cart_order', 'reordered', '_merge', 'price_range_loc'],
      dtype='object')

In [21]:
df_orders_products_merge['orders_day_of_the_week'].value_counts(dropna = False)

orders_day_of_the_week
0    6204182
1    5660230
6    4496490
2    4213830
5    4205791
3    3840534
4    3783802
Name: count, dtype: int64

In [22]:
df_orders_products_merge.shape

(32404859, 18)

### New functions and flags

In [23]:
# New function

result = []

for value in df_orders_products_merge["orders_day_of_the_week"]:
  if value == 0:
    result.append("Busiest day")
  elif value == 4:
    result.append("Least busy")
  else:
    result.append("Regularly busy")
    


In [24]:
df_orders_products_merge['busy_day']=result

In [25]:
df_orders_products_merge['busy_day'].value_counts(dropna = False)

busy_day
Regularly busy    22416875
Busiest day        6204182
Least busy         3783802
Name: count, dtype: int64

## Create and fill new column 'busiest days'

In [26]:
result1 = []

for value in df_orders_products_merge["orders_day_of_the_week"]:
  if value <= 1:
    result1.append("Busiest days")
  elif (value == 3 or value == 4):
    result1.append("Least busy days")
  else:
    result1.append("Regularly busy")
    

In [27]:
df_orders_products_merge['busiest days']=result1

In [28]:
df_orders_products_merge['busiest days'].value_counts(dropna = False)

busiest days
Regularly busy     12916111
Busiest days       11864412
Least busy days     7624336
Name: count, dtype: int64

### These values correspond to the values in 'orders_day_of_the_week' (out[30]) 

### 

## Create and fill new column 'busiest_period_of_day'

In [29]:
df_orders_products_merge['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10    2761760
11    2736140
14    2689136
15    2662144
13    2660954
12    2618532
16    2535202
9     2454203
17    2087654
8     1718118
18    1636502
19    1258305
20     976156
7      891054
21     795637
22     634225
23     402316
6      290493
0      218769
1      115700
5       87961
2       69375
4       53242
3       51281
Name: count, dtype: int64

####  The busiest hours of the day are 10 - 15, the least buses are 0 - 6

In [30]:
result2 = []

for value in df_orders_products_merge['order_hour_of_day']:
  if value <= 6:
    result2.append('Fewest orders')
  elif (value >= 10 and value <= 15):
    result2.append('Most orders')
  else:
    result2.append('Average orders')

In [31]:
df_orders_products_merge['busiest_period_of_day']=result2

In [32]:
df_orders_products_merge['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Most orders       16128666
Average orders    15389372
Fewest orders       886821
Name: count, dtype: int64

## Save as a .pkl file

In [33]:
# Export as .pkl
df_orders_products_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_merged.pkl'))